# Boundary vector score (BVS) - datajoint integration

Datajoint integration of bvs calculation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Set up basics
import sys, os
import numpy as np 
from collections import OrderedDict
# Make plots pretty 
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style='white')
%config InlineBackend.figure_format = 'retina'

import datajoint as dj

In [3]:
# Load base schema
schema = dj.schema(dj.config['dj_imaging.database'])
schema.spawn_missing_classes()

Connecting horsto@kavlidatajoint02.it.ntnu.no:3306


### Schema components

In [4]:
# Load personal schema 
borderscore_schema = dj.schema('user_horsto_borderscore')
borderscore_schema.spawn_missing_classes()

In [17]:
borderscore_schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [16]:
BVScore() & 'bvs>.8'

metasession_name Meta session name (hash),session_order Order of session within meta sessions (zero index!),"session_name Session name: Hash of animal_id, datasource_id, timestamp and combined 'yes'/'no' label",signal_dataset 16 character hash,cell_id Cell ID,channel the channel that this trace comes from (ROI masks are always primary),"spike_filter_id Parameter set ID, starting with A",tracking_dataset 16 character hash,"trackingparams_id Parameter set ID, starting with A","s_t_params_id Parameter set ID, starting with A",signal_type Signal type (Fluorescence vs. Spikes),"map_params_id Parameter set ID, starting with A","field_params_id Parameter set ID, starting with A","bvfield_params_id Parameter set ID, starting with A",bv_field_dect_method Specifies how fields were extracted,bvs Boundary vector score (BVS),orientation
019bb8938659cfee,0,218598b321b72d27,62325d8521fc7d42,1,primary,A,d83d77c1591c8c9f,A,A,spikes,A,A,A,opexebo,0.8235887096774194,horizontal
11aad90b527d9584,2,293b071aad48561a,7ac0cd860549d76e,105,primary,A,9ad82f071ba5bb8e,A,A,df_f,A,A,A,opexebo,0.8412298387096774,horizontal
15a6658964917484,0,8af34258cfc9b26d,47b2cf0720141ef9,94,primary,A,beaa70683858690d,A,A,df_f,A,A,A,opexebo,0.8432459677419355,vertical
